In [1]:
# Imports
from typing import List

import pandas as pd

This notebook will provide: 
 - data explonatory 
 - data insights
 - features selection 
 - transforming dataset into required format (for further modelS applies)

### TODO
- find empty columns <-- there is no missing values

## Assumed steps
#### Benchmark
    - dataset_v0
        # EVALUATE 
        - [catboost_v0] untuned classifier on dataset_v0 data set (~75% accuracy)
        - [catboost_v1] tuned classifier on dataset_v0 data set
        
####  Data set Preprocessing
    - dataset_v1
        - data cleaning
            - ignore the tuples 
            - missing values
                - fill with mean | meadian | custom
            - elinimate noise data 
                - binning method & regression & clustering 
        - data transformation
            - normalization
            - attribute selection
            - discretization
        - data reduction
        

### CAVEATS 
- Imbalanced target (700 vs 300)
- Odd mean loan of creditors age of > 65 


### Steps
1. Rename columns names for explicit visual understanding of features  

In [67]:

class Transformer:
    @staticmethod
    def rename_columns(dataset: pd.DataFrame):
        """Rename dataframe columns names.
        Notes:
            target column names based on provided data_description.txt
        Returns:
            pd.DataFrame - dataframe with renamed columns 
        """
    
        target_columns = {
            'X01': 'account_status',
            'X06': 'account_savings',
            'X02': 'credit_duration',
            'X03': 'credit_history',
            'X04': 'credit_purpose',
            'X05': 'credit_amount',
            'X07': 'employment_status',
            'X17': 'employment_description',
            'X08': 'income_installment_rate',
            'X09': 'gender_status',
            'X10': 'credit_guarantors',
            'X11': 'residence',
            'X12': 'owned_property',
            'X13': 'age',
            'X14': 'installment_plans',
            'X15': 'accomondation_type',
            'X16': 'credit_existing_number',
            'X18': 'liable_maintain',
            'X19': 'phone_number',
            'X20': 'foreign_worker',
            'Y': 'y'
        }
        return dataset.rename(columns=target_columns)    
    
    @staticmethod
    def categorical_columns(dataset: pd.DataFrame):
        """Within dataset find categorical columns
        
        Returns:
          List: list of names of categorical columns.
        """
        categorical_columns_indexes = []
        
        columns = dataset.columns
        num_columns = dataset._get_numeric_data().columns # get numerical columns
        
        categorical_columns = sorted(list(set(columns) - set(num_columns)))
        categorical_columns_indexes = [df.columns.get_loc(column) for column in categorical_columns]
        return categorical_columns, categorical_columns_indexes
        
    
    @staticmethod
    def cast_categorical_column(column: pd.Series):
        pass
    
    @staticmethod
    def train_test_split(dataset: pd.DataFrame, test_size: int, shuffle=False):
        from sklearn.model_selection import train_test_split
        print(f'Splitting dataset {dataset.shape} with test_size :{test_size}')
        X = dataset.drop(columns=['y'])
        y = dataset['y']
        X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=shuffle, test_size=test_size)
        print(f'X_train: {X_train.shape}')
        print(f'y_train: {y_train.shape}')
        print(f'X_test: {X_test.shape}')
        print(f'y_test: {y_test.shape}')
        return X_train, X_test, y_train, y_test
    
    @staticmethod
    def data_cleaning(dataframe: pd.DataFrame):
        """Applies follwoing transformations:   
            - data cleaning
                - ignore the tuples # when the dataset is large and multiple values are missing within a tuple.
                - missing values
                    - fill with mean | meadian | custom
                - encoding
                - elinimate noise data 
                    - binning method & regression & clustering 
        """
        categorical_columns = [column for column in dataset_v1.columns if dataset_v1[f'{column}'].dtype == 'object']
    
        for column_name in categorical_columns: 
            labels = dataframe[f'{column_name}'].astype('category').cat.categories.tolist()
            replace_map_comp = {f'{column_name}' : {k: v for k,v in zip(labels,list(range(1,len(labels)+1)))}}
            dataframe.replace(replace_map_comp, inplace=True)
        return dataframe


    

In [76]:
class Models:
    @staticmethod
    def _catboost_classifier(X_train, X_test, y_train, y_test, cat_features, **kwargs):
        """CatBoost Classifier.
        
        Notes:
          - cat_features: CatBoost model requires list of indexes which denote categorical columns;
        """
        
        from catboost import CatBoostClassifier, Pool
        from sklearn.metrics import accuracy_score
        
        train_data = Pool(
            data=X_train.values,
            label=y_train,
            cat_features=cat_features
        )
        model = CatBoostClassifier(iterations=30)
        model.fit(train_data)
        predictions = model.predict(X_test.values)
        accuracy = accuracy_score(y_test, predictions)
        return accuracy
    
    @staticmethod
    def _xgboost_classifier(X_train, X_test, y_train, y_test, **kwargs):
        """Apply XGBoost Classifier.
        Notes:
          When using XGBoost we need to convert categorical variables into numeric.
        """
        from xgboost import XGBClassifier
        from sklearn.metrics import accuracy_score
        
        model = XGBClassifier()
        model.fit(X_train, y_train, verbose=True)
        predictions = model.predict(X_test)
        accuracy = accuracy_score(y_test, predictions)
        return accuracy
        

## Entry Point

In [79]:
models = Models()
transformer = Transformer()


row_data = pd.read_csv('./dataset/project_data.csv', delimiter=';')
# print(row_data)


# Benchmark
dataset_v0 = transformer.rename_columns(row_data)
categorical_columns, categorical_columns_indexes = transformer.categorical_columns(dataset_v0) 
X_train_v0, X_test_v0, y_train_v0, y_test_v0 = transformer.train_test_split(dataset_v0, test_size=.2, shuffle=True)

# - dataset_v0 && - [catboost_v0] untuned classifier on dataset_v0 data set (~75% accuracy)
dataset_v0_catboost_v0 = models._catboost_classifier(X_train_v0,
                                                     X_test_v0,
                                                     y_train_v0,
                                                     y_test_v0,
                                                     cat_features=categorical_columns_indexes
                                                    )

# - dataset_v0 && - [catboost_v1] tuned classifier on dataset_v0 data set (~__% accuracy)
# @TODO fulfill


# Beating Benchmark
dataset_v1 = transformer.rename_columns(row_data)
dataset_v1 = transformer.data_cleaning(dataset_v1)
X_train_v1, X_test_v1, y_train_v1, y_test_v1 = transformer.train_test_split(dataset_v1, test_size=.2, shuffle=True)

# - dataset_v1 && - [catboost_v0] untuned classifier 
dataset_v1_catboost_v0 = models._catboost_classifier(X_train_v1,
                                                     X_test_v1,
                                                     y_train_v1,
                                                     y_test_v1,
                                                     cat_features=[]
                                                    )

# - dataset_v1 && - [xgboost_classifier_v0] untuned classifier
dataset_v1_xgboost_classifier_v0 = models._xgboost_classifier(X_train_v1,
                                                              X_test_v1,
                                                              y_train_v1,
                                                              y_test_v1
                                                             )






Splitting dataset (1000, 21) with test_size :0.2
X_train: (800, 20)
y_train: (800,)
X_test: (200, 20)
y_test: (200,)
Learning rate set to 0.233364
0:	learn: 0.6137307	total: 4.23ms	remaining: 123ms
1:	learn: 0.5945648	total: 5.8ms	remaining: 81.2ms
2:	learn: 0.5614867	total: 8.6ms	remaining: 77.4ms
3:	learn: 0.5243177	total: 12.2ms	remaining: 79ms
4:	learn: 0.5066539	total: 15.9ms	remaining: 79.5ms
5:	learn: 0.4876814	total: 19.5ms	remaining: 77.9ms
6:	learn: 0.4826167	total: 22.2ms	remaining: 72.9ms
7:	learn: 0.4735014	total: 25.2ms	remaining: 69.4ms
8:	learn: 0.4643574	total: 28.9ms	remaining: 67.5ms
9:	learn: 0.4500081	total: 31.8ms	remaining: 63.6ms
10:	learn: 0.4366963	total: 35.8ms	remaining: 61.8ms
11:	learn: 0.4300341	total: 39.5ms	remaining: 59.2ms
12:	learn: 0.4215742	total: 42.6ms	remaining: 55.7ms
13:	learn: 0.4011467	total: 46ms	remaining: 52.6ms
14:	learn: 0.3883355	total: 49.1ms	remaining: 49.1ms
15:	learn: 0.3776986	total: 52ms	remaining: 45.5ms
16:	learn: 0.3669613	tot

In [80]:
print(dataset_v0_catboost_v0, dataset_v1_catboost_v0, dataset_v1_xgboost_classifier_v0)

0.75 0.725 0.775
